In [1]:
import os
import cv2
import numpy as np
from skimage.util import random_noise

# Specify the indices of the images you want to check
images_to_check = [1, 3, 5]

# Step 1: Get the data from the computer storage
data_dir = r"D:\Z\Data_Set\ttt"  # Replace with the actual path to your dataset

# List of folder names within the data directory
folder_names = ['Crayons', 'Oil Pastel', 'Outline', 'Painting', 'Pencil Shading']

# Step 2: Resize the images
target_size = (229, 229)  # Replace with the desired image size for GoogleNet

# Step 3: Read and process the images in each folder
for folder in folder_names:
    folder_path = os.path.join(data_dir, folder)
    for root, dirs, files in os.walk(folder_path):
        output_dir = os.path.join(root, "Processed_Images")
        os.makedirs(output_dir, exist_ok=True)

        for i, file in enumerate(files):
            if file.lower().endswith(('.jpg', '.png')):
                image_path = os.path.join(root, file)
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Error reading image: {image_path}")
                    continue

                # Step 4: Data normalization and resizing
                resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
                noisy_image = random_noise(resized_image, mode='gaussian', var=0.0001)
                noisy_image = (255 * noisy_image).astype(np.uint8)

                # Step 5: Apply filters
                brightness = np.mean(noisy_image)
                laplacian_var = cv2.Laplacian(noisy_image, cv2.CV_64F).var()
                edges = cv2.Canny(noisy_image, 100, 200)
                edge_pixels_ratio = np.sum(edges) / (edges.shape[0] * edges.shape[1])

                if 20 <= brightness <= 230 and laplacian_var >= 100 and edge_pixels_ratio >= 0.01:
                    # Step 6: Data augmentation
                    enhanced_image = noisy_image  # Use the noisy image as the final processed image

                    # Apply additional augmentation features
                    rotated_image = cv2.rotate(enhanced_image, cv2.ROTATE_90_CLOCKWISE)
                    flipped_image = cv2.flip(enhanced_image, 1)  # Horizontal flip

                    # Save the augmented images in the output directory
                    cv2.imwrite(os.path.join(output_dir, 'augmented_enhanced_' + file), enhanced_image)
                    cv2.imwrite(os.path.join(output_dir, 'augmented_rotated_' + file), rotated_image)
                    cv2.imwrite(os.path.join(output_dir, 'augmented_flipped_' + file), flipped_image)
